In [21]:
from hmm.hmm import HMM

import numpy as np

In [22]:
training_data = np.genfromtxt("../../data/Ex_2.csv", delimiter="," ,dtype=int)[1:, 1:]

In [23]:
gamma = 0.1
beta = 0.2
alpha = 0.5
rates = [1, 5]

# This is uppercase-gamma.
transition_matrix = np.array(
    [[1 - gamma, 0, gamma],
     [0, 1 - gamma, gamma],
     [beta / 2, beta / 2, 1 - beta]]
)

In [24]:
hmm = HMM(transition_matrix, alpha, processing_modes=[0, 1, 2], rates=rates)

In [25]:
num_nodes = 8
time_steps = 50
initial_c = 2

In [26]:
true_processing_modes, true_focus, observations = hmm.forward(
    num_nodes,
    time_steps,
    initial_c,
)

In [27]:
np.array(true_processing_modes)

array([2, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0,
       0, 0, 2, 2, 2, 1])

Dimensions of the joint distribution will be (T - 1, num possible Cs at t, num possible Cs at t +1)

In [28]:
c, z = hmm.nielslief_propagation(observations)

In [29]:
z

array([[[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0.],
        [0.,

> ...it may be more convenient to test the implementation in other ways. You may, for instance, observe that
>
> $$P?(Z_{t,i}=z)-P(Z_{t,i}=z|X=x)$$
>
> has mean 0 and likewise for Ct. Using simulations you can compute such quantities, with $P(Z_{t,i}=z|X=x)$ computed by the inference algorithm, and empirically check if their averages across many replications of the simulations are zero.

In [30]:
num_simulations = 10
correct_C = 0
correct_Z = 0

diff_Z0 = []
diff_Z1 = []

for _ in range(num_simulations):
    true_processing_modes, true_focus, observations = hmm.forward(num_nodes, time_steps, initial_c)
    joint_probabilities_normalised = hmm.infer(observations)
    # Compute the marginal probabilities of C at each time step
    marginal_prob_C = np.sum(joint_probabilities_normalised, axis=2)

    # Calculate the estimated C at each time step
    estimated_C = np.argmax(marginal_prob_C, axis=1)

    # Compute the most likely Z given the estimated C
    estimated_Z = np.zeros((time_steps, num_nodes), dtype=int)

    for t, c in enumerate(estimated_C):
        estimated_Z[t] = hmm.sample_hidden_z(num_nodes, c)

    # Compute the accuracy
    correct_C += np.sum(true_processing_modes[:-1] == estimated_C) / time_steps
    correct_Z += np.sum(true_focus == estimated_Z) / (time_steps * num_nodes)

    # Calculate the marginal probability of Zt,i=0 given X=x
    marginal_prob_Z0_given_X = np.sum(joint_probabilities_normalised[:, :, :2], axis=2)
    # Calculate the marginal probability of Zt,i=1 given X=x
    marginal_prob_Z1_given_X = 1 - marginal_prob_Z0_given_X

    P_Z0 = np.mean(true_focus[:-1] == 0, axis=1)
    P_Z1 = np.mean(true_focus[:-1] == 1, axis=1)

    diff_Z0.append(P_Z0 - np.mean(marginal_prob_Z0_given_X, axis=1))
    diff_Z1.append(P_Z1 - np.mean(marginal_prob_Z1_given_X, axis=1))



AttributeError: 'HMM' object has no attribute 'infer'

In [ ]:
correct_C /= num_simulations
correct_Z /= num_simulations

# Compute the mean differences across all simulations
mean_diff_Z0 = np.mean(np.array(diff_Z0))
mean_diff_Z1 = np.mean(np.array(diff_Z1))

print(f"Proportion of correct C estimations: {correct_C:.2f}")
print(f"Proportion of correct Z estimations: {correct_Z:.2f}")

print(f"Mean difference for Zt,i=0: {mean_diff_Z0:.5f}")
print(f"Mean difference for Zt,i=1: {mean_diff_Z1:.5f}")

Proportion of correct C estimations: 0.49
Proportion of correct Z estimations: 0.71
Mean difference for Zt,i=0: 0.30282
Mean difference for Zt,i=1: -0.30282


We then run the implementation on the real data

In [ ]:
joint_probabilities_normalised = hmm.infer(training_data)
time_steps = joint_probabilities_normalised.shape[0]

In [ ]:
joint_probabilities_normalised.shape

(99, 3, 3)

In [ ]:
# Compute the marginal probabilities of C at each time step
marginal_prob_C = np.sum(joint_probabilities_normalised, axis=2)

# Calculate the estimated C at each time step
estimated_C = np.argmax(marginal_prob_C, axis=1)

# Compute the most likely Z given the estimated C
estimated_Z = np.zeros((time_steps, num_nodes), dtype=int)

for t, c in enumerate(estimated_C):
    estimated_Z[t] = hmm.sample_hidden_z(num_nodes, c)

In [ ]:
estimated_C

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
estimated_Z

array([[0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0,

In [ ]:
joint_probabilities_normalised

array([[[5.58622858e-08, 0.00000000e+00, 6.26489356e-05],
        [0.00000000e+00, 9.21319318e-01, 5.31589801e-03],
        [2.66438797e-07, 5.17876252e-02, 2.15141878e-02]],

       [[4.86880987e-09, 0.00000000e+00, 9.00828803e-06],
        [0.00000000e+00, 9.53672689e-01, 5.46614118e-03],
        [8.95843448e-08, 2.89181210e-02, 1.19339459e-02]],

       [[1.85043829e-09, 0.00000000e+00, 3.07548022e-06],
        [0.00000000e+00, 9.67824454e-01, 5.57776360e-03],
        [6.51745628e-08, 1.87954594e-02, 7.79918020e-03]],

       [[8.52542946e-10, 0.00000000e+00, 1.69508187e-06],
        [0.00000000e+00, 9.73918981e-01, 5.68319728e-03],
        [4.21510736e-08, 1.43619415e-02, 6.03414234e-03]],

       [[2.19563434e-09, 0.00000000e+00, 1.72440925e-06],
        [0.00000000e+00, 9.76266561e-01, 5.85196889e-03],
        [9.53224464e-08, 1.24894097e-02, 5.39023878e-03]],

       [[2.05556694e-09, 0.00000000e+00, 9.31331175e-07],
        [0.00000000e+00, 9.77257655e-01, 6.02181736e-03],
    